# Correlating small-cap pharmaceutical drug companies' stock prices with the FDA drug approval process
CS 198-134: Introduction to Quantitative Finance Final Project

Presented by Cyprian Zander, Enmanual Felix Pena, Jeet Sandhu, Pranay Rajpaul, Shangdian Han

## Abstract

The pharmaceutical industry, often referred to as "Big Pharma," is a crucial sector of the global economy that plays a significant role in advancing healthcare and treating illnesses. The industry's growth and sustainability can be attributed to a steady stream of new drugs and therapies that address a range of diseases and health conditions. However, bringing a new drug to market is a complex and costly process that takes an average of 10-15 years and can cost over $1 billion in research and development. Moreover, the vast majority of drugs fail in the early testing phases, never making it past the preliminary stages and into clinical testing. As a result, significant financial resources must continually be invested in the process of developing new drugs and therapies. This investment is essential for companies to maintain a competitive edge in the industry and to stay ahead of the curve in terms of innovation and discovery.

Pharmaceutical companies, especially smaller ones, are known to have highly volatile stock prices due to a variety of factors. Since the development of new drugs and therapies is a complex and costly process, it is subject to a range of uncertainties, including regulatory hurdles, clinical trial results, and patent expirations. As a result, the stock prices of these companies can be highly sensitive to changes in market sentiment and news related to drug development and approval processes.

Recognizing trends in price fluctuations in pharmaceutical companies could yield a high Sharpe ratio trading strategy. This is because these companies often experience significant price swings in response to news related to their drug development pipelines, clinical trial results, and FDA drug approvals. By identifying patterns in these price movements, investors could potentially develop a trading strategy that generates good returns with low risk.

For example, one significant determinant of price changes in pharmaceutical companies is the FDA drug approval process. The FDA is responsible for evaluating the safety and efficacy of new drugs before they can be marketed and sold to the public. The approval process can have a significant impact on a pharmaceutical company's stock price. If the FDA approves a new drug, it can lead to a significant increase in a company's stock price, as investors perceive the approval as a sign of the drug's potential commercial success. Conversely, if the FDA rejects a new drug, it can lead to a significant decline in a company's stock price, as investors perceive the rejection as a sign of the drug's potential commercial failure.

The highly volatile stock prices of pharmaceutical companies could lead to high Sharpe ratios for specific trading strategies. For instance, a trading strategy that takes advantage of the price movements before and after an FDA drug approval could potentially generate high returns with relatively low risk. By buying the stock before the FDA approval and selling it shortly after the approval, traders could potentially capture the price increase and earn a profit. Similarly, by shorting the stock before the FDA rejection and covering the position after the rejection, traders could potentially capture the price decline and earn a profit.

This report will address how analyzing and leveraging the news and trends within pharmaceutical industry can be an effective way to maximize the Sharpe ratio for trading strategies involving pharmaceutical companies. 

## Research

##### FDA Drug Approval Process

To understand the observations we are tracing to stock price, a general understanding of the full FDA approval process is necessary. This research is taken from drugwatch.com and the fda.gov website.

https://www.drugwatch.com/fda/approval-process/
https://www.fda.gov/patients/drug-development-process/step-3-clinical-research

<br />When wanting to bring a new drug to market, companies must file a **New Drug Application, or NDA**.

**Incentive for pharmaceutical company**
<br />A new patent is good for 20 years from the date the application is filed in the U.S.

**Expedited review process**
<br />Prescription Drug User Fee Act (PDUFA) allows the FDA to collect fees from companies to expedite the drug-approval process.
<br />Standard Review: Drug approval within 10 months.
<br />Priority Review: Drug approval within 6 months.

**Reasons for denying approval**
<br />Unexpected safety issues, manufacturing issues or a failure to prove a drug’s effectiveness.

**The drug approval process**
<br />Basic safety of drug is tested through chemical tests and animal tests.
<br />Clinical trials consist of 3 phases.

1. Phase 1
    - Length: Several months
    - Participants: 20 to 100 healhy volunteers / people with disease
    - Selectivity: 70% pass
    - Intention: Test how drug interacts with human body, adjusting human dosage based on animal data, find out how much body can tolerate and acute side effects, how to best administer the drug to limit risks and maximize benefits
    
    
2. Phase 2
    - Length: Several months to 2 years 
    - Participants: Several hundred people with disease
    - Selectivity: 33% pass
    - Intention: Establish research questions, research methods, and design P3 research protocols, provide additional safety data. Note: this study is still not large enough to show whether drug will help treat the target condition


3. Phase 3
    - Length: 1 to 4 years
    - Participants: 300 to 3000
    - Selectivity: 25-30% pass
    - Intention: Test if it offers a treatment benefit to a specific population, "Pivotal studies", provides most safety data - reveals long term or rare side effects

Passing of phase 3 means FDA approval is given and the drug is released to the market.

4. Phase 4
    - Several thousand volunteers
    - Carried out once drug approved by FDA in _Post Market Safety Monitoring_

**Notes**
<br />Medical Devices have Class I, II, III based on how invasive they are - Pharmaceutical companies are allowed to announce devices before approval process.


As can be seen, the most pivotal stage of the FDA approval process is phase 3, as this approval is followed by bringing the drugs to market and is tje most selective and long process.

## Method

Testing this hypothesis can be broken into the following steps:

1. Collecting small cap pharmaceutical company tickers: find all indexed pharmaceutical stocks, filter by market cap
2. Collect each stock's FDA drug approval history
3. Collect price change data: 1mo at 1hr intervals of stock data following news
4. Analysis: Compare stock performance following news vs. performance of same time frame drawing from a random sample of ticker price history (_or another method of testing the hypothesis?_)

### Importing Libraries

In [32]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 1. Collecting small cap pharmaceutical company tickers

Nasdaq's stock screener (https://www.nasdaq.com/market-activity/stocks/screener) enables filtering of stocks by exchange, market cap, analyst rating, sector, region, and country. We collect tickers from NYSE, NASDAQ, and AMEX by filtering out stocks with market cap > $10B, as these stocks would no longer be considered "small cap", thus falling outside of the parameters of our hypothesis and searching by sector: "Healthcare".

The prepared CSV file now contains 1200 observations, but these also need to be filtered by industry ("Biotechnology: Pharmaceutical Preparations"), as this industry focusses on pharmaceutical drug preparation.

In [15]:
healthcare_stocks = pd.read_csv("nasdaq_screener_1683316923031.csv")

pharmaceutical_preparations = healthcare_stocks.loc[healthcare_stocks["Industry"] == "Biotechnology: Pharmaceutical Preparations"] # Filter by Industry
pharmaceutical_preparations.head()
print("pharmaceutical_preparations contains", len(pharmaceutical_preparations.index), "tickers.")


,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,AADI,Aadi Bioscience Inc. Common Stock,$7.94,0.1600,2.057%,194029701.0,United States,NaN,46995,Health Care,Biotechnology: Pharmaceutical Preparations
3,ABEO,Abeona Therapeutics Inc. Common Stock,$3.32,0.0600,1.84%,59198980.0,United States,NaN,104342,Health Care,Biotechnology: Pharmaceutical Preparations
5,ABOS,Acumen Pharmaceuticals Inc. Common Stock,$5.1675,1.1175,27.593%,211997008.0,United States,2021.0,1167028,Health Care,Biotechnology: Pharmaceutical Preparations
6,ABSI,Absci Corporation Common Stock,$1.40,0.0900,6.87%,129352873.0,United States,2021.0,519172,Health Care,Biotechnology: Pharmaceutical Preparations
8,ABVC,ABVC BioPharma Inc. Common Stock,$0.80,0.0500,6.667%,26464592.0,United States,NaN,36071,Health Care,Biotechnology: Pharmaceutical Preparations


pharmaceutical_preparations contains 619 tickers.



### 2. Collect each stock's FDA drug approval history

Having collected all 619 small-cap pharmaceutical preparation stock tickers, the data to correlate these stocks by is their FDA approval history. 

From the FDA website, we use the past 10 years of drug approval data (see https://www.fda.gov/drugs/development-approval-process-drugs/drug-approvals-and-databases) and match these with our ticker symbols and name collected previously. This requires some data cleaning, as the FDA data has company name abbreviations that do not match the company symbol or name from pharmaceutical_preparations.

Data can only be collected on a by-month basis, thus the data loading is a bit more sophisticated.

Notable about this data is that it does not include phase 1, phase 2, and phase 3 distinctions, but rather only phase 3 approved drugs. An extension to this project could be finding / generating a more rigorous database that also tracks phase 1 and phase 2 approvals.

In [28]:
start_year_incl = 22 # 2022
end_year_excl = 23 # 2023

all_FDA_approvals = pd.DataFrame()

for i in range(start_year_incl, end_year_excl): # year
    for j in range(1, 13): # month
        current_month_FDA = pd.read_csv("./FDA_approvals/%02d%02d.csv" % (j,i,)) #gather data
        all_FDA_approvals = pd.concat([all_FDA_approvals, current_month_FDA], axis=0)

all_FDA_approvals.head()
print("all_FDA_approvals contains", len(all_FDA_approvals.index), "drug approvals.")

,Approval Date,Drug Name,Submission,Active Ingredients,Company,Submission Classification *,Submission Status
0,01/03/2022,"HEPARIN SODIUM 1,000 UNITS AND SODIUM CHLORIDE...",SUPPL-58,HEPARIN SODIUM,BAXTER HLTHCARE,Labeling,Approval
1,01/03/2022,"HEPARIN SODIUM 2,000 UNITS AND SODIUM CHLORIDE...",SUPPL-58,HEPARIN SODIUM,BAXTER HLTHCARE,Labeling,Approval
2,01/03/2022,"HEPARIN SODIUM 5,000 UNITS AND SODIUM CHLORIDE...",SUPPL-58,HEPARIN SODIUM,BAXTER HLTHCARE,Labeling,Approval
3,01/03/2022,NOXAFILNDA #022003,SUPPL-30,POSACONAZOLE,SCHERING,Labeling,Approval
4,01/03/2022,CEFEPIME HYDROCHLORIDEANDA #065441,SUPPL-26,CEFEPIME HYDROCHLORIDE,ACS DOBFAR,Labeling,Approval


all_FDA_approvals contains 4397 drug approvals.


Data cleaning:
 1. Tokenizing company name
 2. Matching stock symbol & name from `pharmaceutical_preparations` with events from `all_FDA_approvals`

## Analysis and Conclusion

_Write text on how to perform analysis_

## Bibliography

_tbd_